# Système de recommandation

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer, QuantileTransformer

In [5]:
df_titre = pd.read_csv("df_less_real.csv").drop(columns = ["Unnamed: 0", "Unnamed: 0.1"])

In [7]:
df = df_titre.drop(columns = ["poster_path", "titre"])

In [9]:
col_num = df.select_dtypes('number').drop(columns = "popularity").columns
col_cat = ["Real"]

In [10]:
col_num

Index(['année', 'temps_minutes', 'note', 'nb_votes'], dtype='object')

In [12]:
df["genre"] = df["genre"].apply(eval)

TypeError: eval() arg 1 must be a string, bytes or code object

In [13]:
df["production_countries"] = df["production_countries"].apply(eval)

## Séparer les colonnes num et cat

Non supervié

In [14]:
multilabelb = MultiLabelBinarizer().fit(df["genre"])
features_cat = multilabelb.transform(df["genre"])
df_cat = pd.DataFrame(features_cat, columns = multilabelb.classes_)

In [15]:
multilabelb_pays = MultiLabelBinarizer().fit(df["production_countries"])
features_cat_pays = multilabelb_pays.transform(df["production_countries"])
df_cat_pays = pd.DataFrame(features_cat_pays, columns = multilabelb_pays.classes_)

In [16]:
quantile_popularity = QuantileTransformer(random_state = 0).fit(df[["popularity"]])
features_popularity = quantile_popularity.transform(df[["popularity"]])
df_popularity = pd.DataFrame(features_popularity, columns=["popularity_transformed"])


In [17]:
col_genre = list(multilabelb.classes_)
col_pays = list(multilabelb_pays.classes_)
col_popularity = ["popularity_transformed"]

In [18]:
df = df.join(df_cat).drop(columns = 'genre').join(df_cat_pays).drop(columns = 'production_countries').join(df_popularity).drop(columns = 'popularity')

In [19]:
df_titre = df_titre.join(df_cat).drop(columns = 'genre').join(df_cat_pays).drop(columns = 'production_countries').join(df_popularity).drop(columns = 'popularity')

In [20]:
#col_num = col_num.drop(columns = "popularity_transformed").columns

C'est l'étape qui va transformer les colonnes numériques (standardisation etc)

In [21]:
numeric_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "mean")), #Remplace les valeurs manquantes
    ("scaling", StandardScaler()) #Standardise 
])

In [22]:
categorical_transformer =  Pipeline(steps = [
    ("encoder", OneHotEncoder())
])

On an cat et num, le porcesseur réunit les 2 

In [23]:
#processeur
preprocessor = ColumnTransformer(
    transformers= [("num", numeric_transformer, col_num),
                   ("cat", categorical_transformer, col_cat),
                   ("unchanged", 'passthrough', list(col_genre) + list(col_pays) + list(col_popularity) )
                   ]
)

In [24]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor), 
    ("model", NearestNeighbors(n_neighbors= 6, metric = 'euclidean')) # Le modèle qu'on veut entraîner 
]
)

In [25]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  Index(['année', 'temps_minutes', 'note', 'nb_votes'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Real']),
                                                 ('unchanged', 'passthrough',
                                                  ['', 'Action', 'Adventure',
                                                   'Animation', 'Biography',
                                                   'Comedy', 'Crime',
                                                   'Documentary', 'Drama',
                                                   'Family', 'Fantasy',
                                                   'Film-Noir', 'History',
                                                   'Horror', 'Music', 'Musical',
                                                   'Mystery', 'News',
                                                   'Reality-TV', 'Romance',
                                                   'Sci-Fi', 'Sport',
                                                   'Talk-Show', 'Thriller',
                                                   'War', 'Western', 'AD', 'AE',
                                                   'AF', 'AL', ...])])),
                ('model', NearestNeighbors(metric='euclidean', n_neighbors=6))])

In [26]:
pipeline.fit(df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  Index(['année', 'temps_minutes', 'note', 'nb_votes'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Real']),
                                                 ('unchanged', 'passthrough',
                                                  ['', 'Action', 'Adventure',
                                                   'Animation', 'Biography',
                                                   'Comedy', 'Crime',
                                                   'Documentary', 'Drama',
                                                   'Family', 'Fantasy',
                                                   'Film-Noir', 'History',
                                                   'Horror', 'Music', 'Musical',
                                                   'Mystery', 'News',
                                                   'Reality-TV', 'Romance',
                                                   'Sci-Fi', 'Sport',
                                                   'Talk-Show', 'Thriller',
                                                   'War', 'Western', 'AD', 'AE',
                                                   'AF', 'AL', ...])])),
                ('model', NearestNeighbors(metric='euclidean', n_neighbors=6))])

In [27]:
data = pd.DataFrame(pipeline.named_steps["preprocessor"].transform(df_titre).toarray())

## Ajouter un poids aux distances

In [28]:
col_names = pipeline.named_steps["preprocessor"].get_feature_names_out()

In [29]:
data.columns = col_names

In [30]:
col_names

array(['num__année', 'num__temps_minutes', 'num__note', ...,
       'unchanged__ZM', 'unchanged__ZW',
       'unchanged__popularity_transformed'], dtype=object)

### Cat

In [31]:
col_real = [real for real in col_names if "Real" in real]
col_genre = [genre for genre in col_names if "genre" in genre]

In [32]:
col_genre

[]

### Dico

Dictionnaire pour attribuer à chaque colonne, un nouveau poids. 

In [33]:
dico_weight = {
    "pop":(["unchanged__popularity_transformed"],1.8),
    "nb_votes":(["num__nb_votes"],2),
    "note":(["num__note"], 1.5),
    "duree":(["num__temps_minutes"], 2),
    #"Real":(col_real, 0.5),
    "genre": (col_genre, 0.2)
    }

In [34]:
pd.set_option("display.max_columns", None)

In [35]:
for nom_f, (liste_cols, pond) in dico_weight.items():
    data[liste_cols] *= pond

In [36]:
data.head()

num__année  num__temps_minutes  num__note  num__nb_votes  \
0   -2.438380           -2.324248  -1.059462      -0.309968   
1   -2.405695           -2.324248   0.845436      -0.310035   
2   -3.092076            0.304431  -0.932469      -0.292667   
3   -2.274955           -1.272776   0.845436      -0.309191   
4   -1.915422           -2.061380   1.607395      -0.309292   

   cat__Real_A. Edward Sutherland  cat__Real_Abby Berlin  \
0                             0.0                    0.0   
1                             0.0                    0.0   
2                             0.0                    0.0   
3                             0.0                    0.0   
4                             0.0                    0.0   

   cat__Real_Abel Ferrara  cat__Real_Abel Gance  cat__Real_Adam Rifkin  \
0                     0.0                   0.0                    0.0   
1                     0.0                   0.0                    0.0   
2                     0.0                   0.0                    0.0   
3                     0.0                   0.0                    0.0   
4                     0.0                   0.0                    0.0   

   cat__Real_Agnieszka Holland  cat__Real_Agnès Varda  cat__Real_Ai Weiwei  \
0                          0.0                    0.0                  0.0   
1                          0.0                    0.0                  0.0   
2                          0.0                    0.0                  0.0   
3                          0.0                    0.0                  0.0   
4                          0.0                    0.0                  0.0   

   cat__Real_Aki Kaurismäki  cat__Real_Akihiko Shiota  cat__Real_Akio Jissôji  \
0                       0.0                       0.0                     0.0   
1                       0.0                       0.0                     0.0   
2                       0.0                       0.0                     0.0   
3                       0.0                       0.0                     0.0   
4                       0.0                       0.0                     0.0   

   cat__Real_Akira Katô  cat__Real_Akira Kurosawa  cat__Real_Akiyoshi Kimata  \
0                   0.0                       0.0                        0.0   
1                   0.0                       0.0                        0.0   
2                   0.0                       0.0                        0.0   
3                   0.0                       0.0                        0.0   
4                   0.0                       0.0                        0.0   

   cat__Real_Al Adamson  cat__Real_Alain Cavalier  cat__Real_Alain Corneau  \
0                   0.0                       0.0                      0.0   
1                   0.0                       0.0                      0.0   
2                   0.0                       0.0                      0.0   
3                   0.0                       0.0                      0.0   
4                   0.0                       0.0                      0.0   

   cat__Real_Alain Resnais  cat__Real_Alain Tanner  cat__Real_Alan Crosland  \
0                      0.0                     0.0                      0.0   
1                      0.0                     0.0                      0.0   
2                      0.0                     0.0                      0.0   
3                      0.0                     0.0                      0.0   
4                      0.0                     0.0                      0.0   

   cat__Real_Alan J. Pakula  cat__Real_Alan James  cat__Real_Alan Parker  \
0                       0.0                   0.0                    0.0   
1                       0.0                   0.0                    0.0   
2                       0.0                   0.0                    0.0   
3                       0.0                   0.0                    0.0   
4                       0.0                   0.0        

### Attribuer une distance

In [37]:
data = pd.DataFrame(pipeline.named_steps["preprocessor"].transform(df_titre).toarray())

In [38]:
dis,index = pipeline.named_steps["model"].kneighbors(data)

In [39]:
pd.DataFrame(index,columns=["source","r1","r2","r3","r4","r5"], index=df_titre["titre"]).to_csv("resultat.csv")

In [40]:
df_titre[["titre", "année", "Real", "poster_path"]].to_csv("df_reco.csv")

Faire un dictionnaire avec {titre:posterpath}


In [41]:
for titre in index[0]:
    display((df_titre.iloc[[titre]]))

,titre,année,temps_minutes,note,nb_votes,Real,poster_path,,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,AD,AE,AF,AL,AM,AN,AO,AQ,AR,AT,AU,AW,AZ,BA,BB,BD,BE,BF,BG,BH,BI,BJ,BM,BO,BR,BS,BT,BW,BY,BZ,CA,CD,CF,CG,CH,CI,CL,CM,CN,CO,CR,CS,CU,CV,CX,CY,CZ,DE,DJ,DK,DO,DZ,EC,EE,EG,EH,ER,ES,ET,FI,FJ,FM,FR,GA,GB,GD,GE,GH,GL,GM,GN,GP,GQ,GR,GT,GW,GY,HK,HN,HR,HT,HU,ID,IE,IL,IN,IQ,IR,IS,IT,JM,JO,JP,KE,KG,KH,KM,KP,KR,KW,KY,KZ,LA,LB,LI,LK,LR,LS,LT,LU,LV,LY,MA,MC,MD,ME,MG,MK,ML,MM,MN,MO,MQ,MR,MS,MT,MU,MW,MX,MY,MZ,NA,NC,NE,NG,NI,NL,NO,NP,NZ,OM,PA,PE,PF,PG,PH,PK,PL,PM,PR,PS,PT,PY,QA,RE,RO,RS,RU,RW,SA,SB,SD,SE,SG,SI,SJ,SK,SL,SM,SN,SO,SR,SS,SU,SV,SY,SZ,TC,TD,TH,TJ,TL,TN,TO,TR,TT,TW,TZ,UA,UG,US,UY,UZ,VA,VE,VG,VN,VU,WS,XC,XG,XI,XK,YE,YU,ZA,ZM,ZW,popularity_transformed
0,The Empress,1917,50.0,5.1,26,peu connu,/5kqaWShpShGdE49DNEOgLESjVsQ.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.367868


,titre,année,temps_minutes,note,nb_votes,Real,poster_path,,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,AD,AE,AF,AL,AM,AN,AO,AQ,AR,AT,AU,AW,AZ,BA,BB,BD,BE,BF,BG,BH,BI,BJ,BM,BO,BR,BS,BT,BW,BY,BZ,CA,CD,CF,CG,CH,CI,CL,CM,CN,CO,CR,CS,CU,CV,CX,CY,CZ,DE,DJ,DK,DO,DZ,EC,EE,EG,EH,ER,ES,ET,FI,FJ,FM,FR,GA,GB,GD,GE,GH,GL,GM,GN,GP,GQ,GR,GT,GW,GY,HK,HN,HR,HT,HU,ID,IE,IL,IN,IQ,IR,IS,IT,JM,JO,JP,KE,KG,KH,KM,KP,KR,KW,KY,KZ,LA,LB,LI,LK,LR,LS,LT,LU,LV,LY,MA,MC,MD,ME,MG,MK,ML,MM,MN,MO,MQ,MR,MS,MT,MU,MW,MX,MY,MZ,NA,NC,NE,NG,NI,NL,NO,NP,NZ,OM,PA,PE,PF,PG,PH,PK,PL,PM,PR,PS,PT,PY,QA,RE,RO,RS,RU,RW,SA,SB,SD,SE,SG,SI,SJ,SK,SL,SM,SN,SO,SR,SS,SU,SV,SY,SZ,TC,TD,TH,TJ,TL,TN,TO,TR,TT,TW,TZ,UA,UG,US,UY,UZ,VA,VE,VG,VN,VU,WS,XC,XG,XI,XK,YE,YU,ZA,ZM,ZW,popularity_transformed
2053,The Risky Road,1918,50.0,5.3,22,peu connu,/uAyxWehKbZibZaOIezrfvPaWG2r.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.325477


,titre,année,temps_minutes,note,nb_votes,Real,poster_path,,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,AD,AE,AF,AL,AM,AN,AO,AQ,AR,AT,AU,AW,AZ,BA,BB,BD,BE,BF,BG,BH,BI,BJ,BM,BO,BR,BS,BT,BW,BY,BZ,CA,CD,CF,CG,CH,CI,CL,CM,CN,CO,CR,CS,CU,CV,CX,CY,CZ,DE,DJ,DK,DO,DZ,EC,EE,EG,EH,ER,ES,ET,FI,FJ,FM,FR,GA,GB,GD,GE,GH,GL,GM,GN,GP,GQ,GR,GT,GW,GY,HK,HN,HR,HT,HU,ID,IE,IL,IN,IQ,IR,IS,IT,JM,JO,JP,KE,KG,KH,KM,KP,KR,KW,KY,KZ,LA,LB,LI,LK,LR,LS,LT,LU,LV,LY,MA,MC,MD,ME,MG,MK,ML,MM,MN,MO,MQ,MR,MS,MT,MU,MW,MX,MY,MZ,NA,NC,NE,NG,NI,NL,NO,NP,NZ,OM,PA,PE,PF,PG,PH,PK,PL,PM,PR,PS,PT,PY,QA,RE,RO,RS,RU,RW,SA,SB,SD,SE,SG,SI,SJ,SK,SL,SM,SN,SO,SR,SS,SU,SV,SY,SZ,TC,TD,TH,TJ,TL,TN,TO,TR,TT,TW,TZ,UA,UG,US,UY,UZ,VA,VE,VG,VN,VU,WS,XC,XG,XI,XK,YE,YU,ZA,ZM,ZW,popularity_transformed
1132,Redeeming Love,1916,50.0,5.0,26,peu connu,/9x5dAeJL9LgRkHdw2XnXh75o7IL.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.17017


,titre,année,temps_minutes,note,nb_votes,Real,poster_path,,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,AD,AE,AF,AL,AM,AN,AO,AQ,AR,AT,AU,AW,AZ,BA,BB,BD,BE,BF,BG,BH,BI,BJ,BM,BO,BR,BS,BT,BW,BY,BZ,CA,CD,CF,CG,CH,CI,CL,CM,CN,CO,CR,CS,CU,CV,CX,CY,CZ,DE,DJ,DK,DO,DZ,EC,EE,EG,EH,ER,ES,ET,FI,FJ,FM,FR,GA,GB,GD,GE,GH,GL,GM,GN,GP,GQ,GR,GT,GW,GY,HK,HN,HR,HT,HU,ID,IE,IL,IN,IQ,IR,IS,IT,JM,JO,JP,KE,KG,KH,KM,KP,KR,KW,KY,KZ,LA,LB,LI,LK,LR,LS,LT,LU,LV,LY,MA,MC,MD,ME,MG,MK,ML,MM,MN,MO,MQ,MR,MS,MT,MU,MW,MX,MY,MZ,NA,NC,NE,NG,NI,NL,NO,NP,NZ,OM,PA,PE,PF,PG,PH,PK,PL,PM,PR,PS,PT,PY,QA,RE,RO,RS,RU,RW,SA,SB,SD,SE,SG,SI,SJ,SK,SL,SM,SN,SO,SR,SS,SU,SV,SY,SZ,TC,TD,TH,TJ,TL,TN,TO,TR,TT,TW,TZ,UA,UG,US,UY,UZ,VA,VE,VG,VN,VU,WS,XC,XG,XI,XK,YE,YU,ZA,ZM,ZW,popularity_transformed
1143,The Wood Nymph,1916,50.0,5.0,14,peu connu,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.115115


,titre,année,temps_minutes,note,nb_votes,Real,poster_path,,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,AD,AE,AF,AL,AM,AN,AO,AQ,AR,AT,AU,AW,AZ,BA,BB,BD,BE,BF,BG,BH,BI,BJ,BM,BO,BR,BS,BT,BW,BY,BZ,CA,CD,CF,CG,CH,CI,CL,CM,CN,CO,CR,CS,CU,CV,CX,CY,CZ,DE,DJ,DK,DO,DZ,EC,EE,EG,EH,ER,ES,ET,FI,FJ,FM,FR,GA,GB,GD,GE,GH,GL,GM,GN,GP,GQ,GR,GT,GW,GY,HK,HN,HR,HT,HU,ID,IE,IL,IN,IQ,IR,IS,IT,JM,JO,JP,KE,KG,KH,KM,KP,KR,KW,KY,KZ,LA,LB,LI,LK,LR,LS,LT,LU,LV,LY,MA,MC,MD,ME,MG,MK,ML,MM,MN,MO,MQ,MR,MS,MT,MU,MW,MX,MY,MZ,NA,NC,NE,NG,NI,NL,NO,NP,NZ,OM,PA,PE,PF,PG,PH,PK,PL,PM,PR,PS,PT,PY,QA,RE,RO,RS,RU,RW,SA,SB,SD,SE,SG,SI,SJ,SK,SL,SM,SN,SO,SR,SS,SU,SV,SY,SZ,TC,TD,TH,TJ,TL,TN,TO,TR,TT,TW,TZ,UA,UG,US,UY,UZ,VA,VE,VG,VN,VU,WS,XC,XG,XI,XK,YE,YU,ZA,ZM,ZW,popularity_transformed
2694,His Birthright,1918,50.0,5.3,42,peu connu,/gIsPhWNwRCjwMDGZgJMQgmewf82.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.147147


,titre,année,temps_minutes,note,nb_votes,Real,poster_path,,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,AD,AE,AF,AL,AM,AN,AO,AQ,AR,AT,AU,AW,AZ,BA,BB,BD,BE,BF,BG,BH,BI,BJ,BM,BO,BR,BS,BT,BW,BY,BZ,CA,CD,CF,CG,CH,CI,CL,CM,CN,CO,CR,CS,CU,CV,CX,CY,CZ,DE,DJ,DK,DO,DZ,EC,EE,EG,EH,ER,ES,ET,FI,FJ,FM,FR,GA,GB,GD,GE,GH,GL,GM,GN,GP,GQ,GR,GT,GW,GY,HK,HN,HR,HT,HU,ID,IE,IL,IN,IQ,IR,IS,IT,JM,JO,JP,KE,KG,KH,KM,KP,KR,KW,KY,KZ,LA,LB,LI,LK,LR,LS,LT,LU,LV,LY,MA,MC,MD,ME,MG,MK,ML,MM,MN,MO,MQ,MR,MS,MT,MU,MW,MX,MY,MZ,NA,NC,NE,NG,NI,NL,NO,NP,NZ,OM,PA,PE,PF,PG,PH,PK,PL,PM,PR,PS,PT,PY,QA,RE,RO,RS,RU,RW,SA,SB,SD,SE,SG,SI,SJ,SK,SL,SM,SN,SO,SR,SS,SU,SV,SY,SZ,TC,TD,TH,TJ,TL,TN,TO,TR,TT,TW,TZ,UA,UG,US,UY,UZ,VA,VE,VG,VN,VU,WS,XC,XG,XI,XK,YE,YU,ZA,ZM,ZW,popularity_transformed
871,The Vixen,1916,60.0,5.0,19,peu connu,/4tNwEXgWo7A5GVrViEdKHwKA1OH.jpg,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.198699


## Matrice